In [2]:
%%html
<div id="canny_final_container" style="text-align: center; background: #1a1a1a; padding: 25px; border-radius: 20px; color: white; font-family: sans-serif;">
    <h2 style="color: #3498db;">Canny Edge: Detección de Bordes</h2>
    <p style="font-size: 13px; color: #aaa;">Usa derivadas y diferencias finitas para encontrar contornos.</p>
    
    <div style="position: relative; display: inline-block; margin-bottom: 20px;">
        <video id="v_canny_raw" width="640" height="480" style="display:none" playsinline></video>
        <canvas id="c_canny_final" width="640" height="480" style="background: #000; border: 3px solid #333; border-radius: 15px;"></canvas>
    </div>
    
    <div style="margin-top: 10px;">
        <button id="btn_on_canny" onclick="forzarCannyAbsoluto()" style="padding: 15px 30px; background: #27ae60; color: white; border: none; border-radius: 10px; cursor: pointer; font-weight: bold; font-size: 16px;">
            🚀 ACTIVAR CANNY AQUÍ
        </button>
        <button id="btn_off_canny" onclick="apagarCannyAbsoluto()" style="padding: 15px 30px; background: #e74c3c; color: white; border: none; border-radius: 10px; cursor: pointer; font-weight: bold; font-size: 16px; margin-left: 10px; display: none;">
            🛑 APAGAR
        </button>
    </div>
    <p id="msg_canny_final" style="color: #f1c40f; font-size: 14px; margin-top: 15px;">Estado: Esperando acción del usuario.</p>
</div>

<script async src="https://docs.opencv.org/4.8.0/opencv.js" type="text/javascript"></script>

<script>
// Referencias únicas para evitar choques entre celdas
var vidCanny = document.getElementById('v_canny_raw');
var canCanny = document.getElementById('c_canny_final');
var txtStatus = document.getElementById('msg_canny_final');
var btnOn = document.getElementById('btn_on_canny');
var btnOff = document.getElementById('btn_off_canny');

var cannyRunning = false;
var cannyStream = null;

async function forzarCannyAbsoluto() {
    txtStatus.innerText = "Limpiando conexiones previas...";
    
    // 1. LIMPIEZA TOTAL: Matamos cualquier cámara abierta en la ventana
    try {
        if (window.currentStream) {
            window.currentStream.getTracks().forEach(t => t.stop());
            window.currentStream = null;
        }
    } catch(e) { console.log("Limpio."); }

    // 2. FORZAR INICIO
    try {
        cannyStream = await navigator.mediaDevices.getUserMedia({ video: true });
        window.currentStream = cannyStream; // Compartimos el stream para que otros códigos puedan cerrarlo
        vidCanny.srcObject = cannyStream;
        vidCanny.play();
        
        btnOn.style.display = 'none';
        btnOff.style.display = 'inline-block';
        txtStatus.innerText = "Cámara forzada con éxito. Procesando...";
        
        cannyRunning = true;
        iniciarIA_Canny();
    } catch (err) {
        txtStatus.innerText = "Error: No se pudo forzar la cámara. Revisa permisos.";
        console.error(err);
    }
}

function apagarCannyAbsoluto() {
    cannyRunning = false;
    if (cannyStream) {
        cannyStream.getTracks().forEach(track => track.stop());
    }
    vidCanny.pause();
    vidCanny.srcObject = null;
    btnOn.style.display = 'inline-block';
    btnOff.style.display = 'none';
    txtStatus.innerText = "Cámara liberada y apagada.";
}

function iniciarIA_Canny() {
    if (!cannyRunning) return;

    // Inicializamos matrices OpenCV (C++ compilado a WebAssembly)
    let src = new cv.Mat(vidCanny.height, vidCanny.width, cv.CV_8UC4);
    let dst = new cv.Mat(vidCanny.height, vidCanny.width, cv.CV_8UC1);
    let cap = new cv.VideoCapture(vidCanny);

    function procesarFrame() {
        if (!cannyRunning) {
            src.delete(); dst.delete();
            return;
        }
        
        try {
            cap.read(src); // Leer frame del video
            
            // Paso 1: Escala de Grises (Requisito fundamental de Canny)
            cv.cvtColor(src, dst, cv.COLOR_RGBA2GRAY); 
            
            // Paso 2: Algoritmo Canny (Detecta bordes con derivadas)
            // Umbrales 50 y 150 para conectar bordes
            cv.Canny(dst, dst, 50, 150, 3, false); 
            
            cv.imshow('c_canny_final', dst); // Mostrar resultado
            requestAnimationFrame(procesarFrame);
        } catch (err) {
            console.error("Error en loop:", err);
            cannyRunning = false;
        }
    }
    requestAnimationFrame(procesarFrame);
}
</script>